# Predicting recurrence of thyroid cancer

<span style="font-size:14px">
Andre Nana

### 1. Introduction

<span style="font-size:14px">

#### Background

Thyroid cancer is one of the most common endocrine malignancies and is on the rise in both the US and globally [^1] This cancer generally has a favorable prognosis but still carries a significant risk of recurrence. Traditional statistical methods are commonly used to assess the relationships between clinical, pathological, and demographic predictors and thyroid cancer recurrence, but usually rely on predefined assumptions about variable relationships (independence of observations, homoscedasticity, etc.). Machine learning (ML) techniques can provide more flexible approaches by adaptively learning patterns directly from the data. This can enable more accurate predictions, especially in complex, high-dimensional clinical datasets, and more importantly, allow prediction on new data.

#### Purpose
  
This project investigates recurrence in thyroid cancer patients using both statistical logistic regression and machine learning methods. With the logistic regression, we explore the relationship between several “causes” and the likelihood of recurrence. We then shift to machine learning techniques for risk stratification and evaluate the predictive accuracy of the following algorithms: 

- logistic regression,
- K-Nearest Neighbors (KNN),
- Support Vector Machines (SVM),
- Decision Tree,
- Random Forest, and,
- Artificial Neural Networks (ANN)

These algorithms are evaluated based on the following performance metrics:

-	Sensitivity (ability to correctly identify patients who experience recurrence), 
-	Specificity (ability to correctly identify those who do not), 
-	Positive Predictive Value (PPV) (the probability that predicted recurrences are true), 
-	Negative Predictive Value (NPV) (the probability that predicted non-recurrences are true),
-	Area Under the ROC curve (AUC) (overall discriminatory ability),
-	Accuracy (the proportion of correct classifications across all cases).

#### Data Source and initial study

The dataset in this study is publicly accessible and was generously provided by Borzooei et al., who published their original findings in 2024 [^2]. Thyroid cancer patients were followed prospectively for at least 10 years, to assess whether their cancer recurred, and  several demographic, clinical and pathological measures were collected. In total 383 patients were included, which is a relatively small sample size. A full description of the study design can be consulted in the published paper. While the study is longitudinal in design, it does not provide  time-to-event information which prevents the use of appropriate ML techniques tailored for longitudinal data [^3]. The lack of this crucial piece of information coerces us to focus instead on models that apply to binary outcomes, such as those cited above. 

#### Disclaimer
  
This project does not aim to replicate the original study by Borzooei et al. Rather, it demonstrates alternative analytic strategies for examining the same problem. Given these differences in methodological choices and assumptions, results were not expected to mirror those of the original study. 
This analysis is intended for educational and research purposes only and has not been peer-reviewed. While efforts have been made to ensure the accuracy of the methods and results, the author does not guarantee the correctness or completeness of the analysis. The author bears no responsibility or liability for any errors, omissions, or outcomes resulting from the use of this material. Use at your own discretion.



[^1]: Kitahara CM, Schneider AB. Epidemiology of Thyroid Cancer. Cancer Epidemiol Biomarkers Prev. 2022 Jul 1;31(7):1284-1297. doi: 10.1158/1055-9965.EPI-21-1440. PMID: 35775227; PMCID: PMC9473679.

[^2]: Borzooei S, Briganti G, Golparian M, Lechien JR, Tarokhian A. Machine learning for risk stratification of thyroid cancer patients: a 15-year cohort study. Eur Arch Otorhinolaryngol. 2024 Apr;281(4):2095-2104. doi: 10.1007/s00405-023-08299-w. Epub 2023 Oct 30. PMID: 37902840.

[^3]: Cascarano, A., Mur-Petit, J., Hernández-González, J. et al. Machine and deep learning for longitudinal biomedical data: a review of methods and applications. Artif Intell Rev 56 (Suppl 2), 1711–1771 (2023). https://doi.org/10.1007/s10462-023-10561-w

### 2. Loading libraries and data

 #### 2.1 Import libraries

In [ ]:
# =========================
# Core libraries
# =========================
import warnings

import numpy as np
import pandas as pd

# =========================
# Visualization
# =========================
import matplotlib.pyplot as plt
import plotly.express as px

# =========================
# Statistics & Modeling
# =========================
import statsmodels.formula.api as smf

# =========================
# Machine Learning (scikit-learn)
# =========================

# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Calibration
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc, precision_recall_curve, brier_score_loss
)

# =========================
# Other analytical tools
# =========================
from tableone import TableOne

# =========================
# Import UCI ML repository
# =========================

from ucimlrepo import fetch_ucirepo 

# =========================
# Suppress warnings
# =========================
warnings.filterwarnings("ignore")


#### 2.1 Import dataset

In [ ]:
# fetch dataset 
differentiated_thyroid_cancer_recurrence = fetch_ucirepo(id=915) 
# data (as pandas dataframes) 
X = differentiated_thyroid_cancer_recurrence.data.features 
y = differentiated_thyroid_cancer_recurrence.data.targets 
  
# metadata 
# print(differentiated_thyroid_cancer_recurrence.metadata) 
# variable information 
# print(differentiated_thyroid_cancer_recurrence.variables) 
# combine X and y into a single dataframe
tcr_original = pd.concat([X, y], axis=1)

### 3. Data Exploration

#### 3.1 Overview of the data

In [ ]:
tcr_raw = tcr_original.copy() 
tcr_raw.head()

In [ ]:
print(tcr_raw.info())

<span style="font-size:14px">
We have 17 variables and 383 observations. We do not have any missing values


<span style="font-size:14px">

Variables description (see Borzooei et al. 2024 for more details):



<strong><em>Demographics &amp; History</em></strong>

- Age at diagnosis (in years)
- Biological sex (Female, Male)
- Current smoking status
- Past smoking history
- History of radiation therapy to head and neck region

<strong><em>Thyroid Characteristics</em></strong>

- Thyroid function: euthyroid, clinical/subclinical hypothyroidism or hyperthyroidism
- Presence of goiter:diffuse, single nodular (left/right lobe), multinodular, normal

<strong><em>Physical Exam Findings</em></strong>

- Presence of adenopathy on physical exam: no adenopathy, anterior right, anterior left, bilateral, posterior, extensive (all locations)

<strong><em>Cancer Pathology</em></strong>

- Pathological subtype of cancer: papillary, micropapillary, follicular, Hürthle cell
- Focality:unifocal, multifocal

<strong><em>Staging & Risk</em></strong>

- ATA risk assessment: low, intermediate, high
- TNM staging: T score (Tumor size/local invasion),N score (Lymph node involvement), M score (Distant metastasis)
- Final stage

<strong><em>Treatment & Outcome</em></strong>

- Initial treatment response: excellent, biochemical incomplete, structurally incomplete, indeterminate
- Recurrence status: locoregional recurrence, distant metastasis

### 3.2 Unvariate data exploration

- Continouous variable (age at diagnosis)

In [ ]:
plt.figure(figsize=(7, 6))
plt.hist(tcr_raw["Age"], bins=30, color="#1b557c", edgecolor="white")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.title("Histogram of Age")
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()


In [ ]:
tcr_raw["Age"].describe().round(2)

<span style="font-size:14px">
Age ranges from 15 to 82 with a mean of 40.87 and a median of 37 (std=15.13, IQR=22)

In [ ]:
from scipy.stats import skew, kurtosis

skew_age = skew(tcr_raw["Age"], nan_policy="omit").round(2)
kurt_age = kurtosis(tcr_raw["Age"], nan_policy="omit").round(2)

print("Skewness:", skew_age)
print("Kurtosis:", kurt_age)


<span style="font-size:14px">
Age variable is moderately right-skewed (younger cluster, fewer older outliers) and a bit flatter than normal.

<span style="font-size:14px">
we will create a Age groups based on American Thyroid Association (ATA) risk stratification guidelines

https://doi.org/10.1089/thy.2019.0688

In [ ]:
tcr_raw['Age category'] = pd.cut(tcr_raw['Age'], 
                                 bins=[0, 55, 100], # max age is 62
                                 labels=['0-55', '55+'], 
                                 right=False)

- Categorical variables

We decide to create a new variable for smoking status that combines current smoking status and past smoking history.

In [ ]:
import matplotlib.pyplot as plt

# List of categorical variables to plot
cat_vars = [
    'Recurred', 'Age category', 'Gender', 'Smoking', 'Hx Smoking', 
    'Hx Radiothreapy', 'Thyroid Function', 'Physical Examination',
    'Adenopathy', 'Pathology', 'Focality', 'Risk', 'T', 'N', 'M',
    'Stage', 'Response'
]

# Number of plots
n_vars = len(cat_vars)

# Create figure with 2 columns
fig, axes = plt.subplots(nrows=(n_vars + 1) // 2, ncols=2, figsize=(12, 4 * ((n_vars + 1) // 2)))
axes = axes.flatten()  # flatten for easy iteration

# Plot each categorical variable
for i, var in enumerate(cat_vars):
    tcr_raw[var].value_counts().plot(
        kind="bar",
        ax=axes[i],
        color="#1b557c",
        edgecolor="white"
    )
    axes[i].set_title(var)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("Count")
    axes[i].spines["top"].set_visible(False)
    axes[i].spines["right"].set_visible(False)

# Hide unused subplots if odd number of vars
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


<span style="font-size:14px">
We observed that the majority of individuals did not experience recurrence compared to those who did. The cohort was characterized by a higher proportion of participants younger than 55, more females, non-smokers and non-current smokers, individuals without a history of radiotherapy, those with normal thyroid function, no evidence of adenopathy, and overall a lower-risk profile

### 3.3 Assessing multicollinearity among predictors

In [ ]:
mm = pd.get_dummies(tcr_raw.drop(['Age', 'Recurred'], axis=1), drop_first=True)
corr_matrix = mm.corr()

In [ ]:
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
corr_masked = corr_matrix.mask(mask)

fig = px.imshow(
    corr_masked,
    text_auto=".2f",  
    color_continuous_scale=["#ebf2f6", "#aed6f1", "#3498db"],
    aspect="auto"
)

fig.update_layout(
    width=1000,
    height=900,
    font=dict(size=8, color="#1b557c"),       
    xaxis=dict(
        tickangle=-45,
        showticklabels=True,
        showgrid=True,
        zeroline=False
    ),
    yaxis=dict(
        showticklabels=True,
        showgrid=True,
        zeroline=False
    ),
    title="Correlation Heatmap",
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

<span style="font-size:14px">
From this correlation matrix, we can see that there is strong correlation between for instance

- low risk (ATA) and no adenopathy with a pearson correlation coefficient of 0.68
- low risk (ATA) and response excellent with a pearson correlation coefficient of 0.60
- Stage IVB and M1 with a pearson correlation coefficient of 0.70


<span style="font-size:14px">
In the code below, we create a code to identify in the  pairwise manner, the correlation among predictors of recurrence of thyroid cancer.

In [ ]:
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
filtered = corr_matrix.where(mask)

strong_corr = filtered.stack().reset_index()
strong_corr.columns = ['Feature 1', 'Feature 2', 'Correlation']

threshold = 0.6
strong_corr = strong_corr[strong_corr['Correlation'].abs() > threshold]

strong_corr = strong_corr.reindex(strong_corr['Correlation'].abs().sort_values(ascending=False).index)

print(strong_corr)


<span style="font-size:14px">
We identify the same correlated variables such as low risk and no adenopathy, metastasis and stage IVB etc. Therefore, it is the right time to discuss about cancer classification and what is conveyed by these variables.

What is tumor staging?
Tumor staging conveys information about the size of the tumor and the extend to which it has spread in the body. It is a clinical management tool, since a localized tumor could benefit from simple resection with an appropriate follow-up,whereas metastasized tumors would require surgery and aggressive adjuvant therapies such as radiation or chemotherapy among others.
Staging may differ between cancer types, but the TNM system is the most widely used. For thyroid cancer for instance, staging is based on the American Joint Committee on Cancer (AJCC) Tumor, Node, Metastasis (TNM) system.
- T describes the size of the original (primary) tumor and whether it has invaded nearby tissue,
- N describes nearby (regional) lymph nodes that are involved,
- M describes distant metastasis (spread of cancer from one part of the body to another).
  
So, basically, staging is some form of summarizing the information contained in T, N, and M scores, for clinical management purposes. Similarly, an adenopathy is just a lymph node involvement (N score). Therefore, it is expected that there is a correlation between adenopathy and N score. At the physical examination, the physician will assess the size of the tumor wich is part of the T score in the TNM system.

We can already see that having the stage itself forfeits the need for T, N, M scores as well as adenopathy and physical examination. Likewise, the ATA risk stratification is a summary of several variables including the type, differentiation, local and regional invasion an distant metastis among other.

From a machine learning perspective, algorithms vary in their sensitivity to multicollinearity. Linear and logistic regression are highly susceptible, as strong correlations among predictors can destabilize coefficient estimates and hinder interpretation. In contrast, tree-based methods such as decision trees and random forests are generally more resilient to multicollinearity. Nonetheless, even in these models, multicollinearity may still reduce the interpretability of results by obscuring the relative importance of individual features.

We will run different sets of models with or without correlated variables.


<span style="font-size:14px">
For the purpose of the statistical logistic regression, we decide to collapse some categories, which conveys more meaningful information and avoid issues with small sample sizes in some categories.

In [ ]:
# Smoking status: Yes vs. No
tcr_raw['Smoking status'] = tcr_raw.apply(
    lambda row: "Yes" 
                if (row['Smoking'] == "Yes" or row['Hx Smoking'] == "Yes") 
                else "No",
    axis=1
)

# Thyroid function: Euthyroid vs. Else
tcr_raw['Thyroid_01'] = (
    tcr_raw['Thyroid Function']
    .astype(str).str.strip().str.lower()
    .map(lambda x: 'euthyroid' if x == 'euthyroid' else 'non-euthyroid')
)

# Physical Examination: Normal vs. Else
tcr_raw['Physical_01'] = (
    tcr_raw['Physical Examination']
    .astype(str).str.strip().str.lower()
    .map(lambda x: 'Normal' if x == 'normal' else 'goiter')
    )

# Adenopathy: Yes vs. No
tcr_raw['Adenopathy_01'] = (
    tcr_raw['Adenopathy']
    .astype(str).str.strip().str.lower()
    .map(lambda x: 'No' if x == 'no' else 'yes')
    )

# Response: Excellent vs. Else
tcr_raw['Response_01'] = (
    tcr_raw['Response']
    .astype(str).str.strip().str.lower()
    .map(lambda x:  'excellent' if x == 'excellent' else 'not-excellent')
)

#### 3.4 Distribution of features by recurrence status

In [ ]:
columns = ['Age', 'Age category', 'Gender', 'Smoking status',
           'Smoking', 'Hx Smoking', 'Thyroid Function','Thyroid_01', 
           'Physical Examination', 'Physical_01', 'Adenopathy', 'Adenopathy_01',        
           'Hx Radiothreapy', 'Pathology', 'Focality', 
           'Risk', 'T', 'N', 'M', 'Stage', 'Response', 'Response_01']

categorical = ['Age category', 'Gender', 'Smoking status',
           'Smoking', 'Hx Smoking', 'Thyroid Function','Thyroid_01', 
           'Physical Examination', 'Physical_01', 'Adenopathy', 'Adenopathy_01',        
           'Hx Radiothreapy', 'Pathology', 'Focality', 
           'Risk', 'T', 'N', 'M', 'Stage', 'Response', 'Response_01']
table1 = TableOne(data=tcr_raw, columns=columns, categorical=categorical,
                  groupby='Recurred', pval=True)
print(table1) 

<span style=font-size:14px>
As expected, compared to individuals without recurrence, those with recurrence were more likely to be older (39.8% aged over 55 years versus 12.7%), to have a current or past history of smoking (34.3% versus 10.2%), and to present with more advanced cancer stages (stage II or higher). Nearly all individuals with recurrence had a non-excellent response to initial treatment (99.1% versus 24.7%).

### 4. Modeling

### 4.1 Statistical logistic regression

### 4.1.1 Problem statement

<span style=font-size:14px>
In traditional statistical research, we would explore the relationship between one or more predictors and a response variable, sometimes from a causal perspective. This implies formulating a research question and proceeding with the hypothesis testing. In this particular project, we could hypothesize that:

- Null hypothesis: patients who had recurrence of thyroid cancer do not differ from those who did not have recurrence with respect to whether they ever smoked or not.
- Alternative hypothesis: patients who had recurrence of thyroid cancer differ from those who did not have recurrence with respect to whether they ever smoked or not.

Note that we would use the whole dataset and produce parameter estimates and p-values. We will evaluate how good our model fits the data at hand, and identify the signigicance of the association (probably from a causal perspective since we have a prospective cohort) between smoking and reecurrence in this case. For instance, we may find that people who smoke or had a history of smoking, have higher odds of recurrence compared to people who never smoked. However, with this approach we would not be able to make predictions on new data.

<span style=font-size:14px>
As mentioned earlier, we exclude some variables from the model to avoid multicollinearity. We will consider smoking status (ever/never smoked), as our exposure of insterest, and hypothesize that people who ever smoked have higher odds of recurrence compared to people who never smoked, adjusting for other predictors. 

We will run several models with different sets of predictors. As shown in Table 1, several predictors exhibit sparse or highly imbalanced categories that create quasi-complete or perfect separation, such as Response (with “Excellent” almost always non-recurrent and “Structural Incomplete” almost always recurrent), advanced T stages (T4b exclusively recurrent, T4a and T3b heavily skewed), and M stage (M1 always recurrent), all of which can lead to convergence issues in logistic regression. The observation is true for Stage where all stages higher or equal to stage III recurred. Moreover, some variables are highly correlated, such as Stage with T, N, M scores and adenopathy and risk stratification which is a composite of several variables. We will therefore exclude these variables from our models.

In [ ]:
# Formatting for logistic regression
tcr_log0 = tcr_raw.copy()
tcr_log0['Recurred_01'] = (
    tcr_log0['Recurred']
    .astype(str).str.strip().str.lower()
    .map({'yes': 1, 'no': 0})
)

tcr_log1 = tcr_log0.rename(columns={
    "Age category": "Age_category",
    "Smoking status": "Smoking_status",
    "Hx Radiothreapy": "Hx_Radiotherapy",
    "Physical Examination": "Physical_Examination"  
})

tcr_log = tcr_log1.drop(columns=['Age', 'Smoking', 'Hx Smoking', 'Thyroid Function', 'Response', 
                                 'Response_01', 'T', 'N', 'M', 'Stage', 'Adenopathy', 'Physical_Examination', "Risk"])

### 4.1.1 Distribution of variables by EXPOSURE status

In [ ]:
tcr_log.columns

In [ ]:
columns = ['Recurred', 'Age_category', 'Gender', 'Thyroid_01', 'Physical_01', 'Adenopathy_01',
           'Hx_Radiotherapy', 'Pathology', 'Focality']

categorical = ['Recurred', 'Age_category', 'Gender', 'Thyroid_01', 
               'Physical_01','Hx_Radiotherapy', 'Pathology', 'Focality', 
       
       'Adenopathy_01']
table_log = TableOne(data=tcr_log, columns=columns, categorical=categorical,
                  groupby='Smoking_status', pval=True)
print(table_log ) 

In [ ]:
tcr_raw.columns

<span style=font-size:14px>

From this table, we can see that more than half (56.9%) of people who ever smoked had recurrence, whereas 22.3% (71 out of 247) of people who never smoked had recurrence. 

<span style=font-size:14px>
The stepwise approach in regression modeling is considered outdated and generally not advised. One should build a causal diagram in the form of a Directed Acyclic Graph (DAG) to identify confounders, mediators, effect measure modifiers, and colliders, and decide which variables to include in the model based on this causal framework. We will adopt both approaches here with first a DAG-based model, then a stepwise approach.


### 4.1.3 Causal diagram (DAG) for recurrence of thyroid cancer

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define DAG
G = nx.DiGraph()
edges = [
    ("Smoking", "Recurrence"),
    ("Age", "Smoking"),
    ("Age", "Recurrence"),
    ("Sex", "Smoking"),
    ("Sex", "Recurrence"),
    ("Thyroid function", "Recurrence"),
    ("Physical examination", "Recurrence"),
    ("Pathology", "Recurrence"),
    ("Focality", "Recurrence"),
    ("TNM, Stage, Response, Risk", "Recurrence"),
    ("Recurrence", "History of Radiotherapy")
]
G.add_edges_from(edges)

# Node groups
exposure = ["Smoking"]
outcome = ["Recurrence"]
confounders = ["Age", "Sex"]
instrumental = ["Thyroid function", "Physical examination", "Pathology", "Focality"]
collider = ["History of Radiotherapy"]
excluded = ["TNM, Stage, Response, Risk"]

# Assign colors
node_colors = []
for node in G.nodes():
    if node in exposure:
        node_colors.append("orange")   # exposure
    elif node in outcome:
        node_colors.append("red")      # outcome
    elif node in confounders:
        node_colors.append("lightgreen")  # confounders
    elif node in instrumental:
        node_colors.append("lightblue")  # instrumental variables
    elif node in collider:
        node_colors.append("orange")  # collider
    elif node in excluded:
        node_colors.append("purple")  # excluded
    else:
        node_colors.append("white")  # fallback

# Manual layout (x, y coordinates)
pos = {
    "Smoking": (0, 0),
    "Age": (1, 5),
    "Sex": (1, 3.5),
    "Thyroid function": (1, 2),
    "Physical examination": (1, 0.7),
    "Pathology": (1, -1),
    "Focality": (1, -2.5),
    "Recurrence": (2, 0),
    "History of Radiotherapy": (2, -2),
    "TNM, Stage, Response, Risk": (2, 4)
}

# Draw DAG
plt.figure(figsize=(10,6))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=4000,
    node_color=node_colors,
    font_size=10,
    arrowsize=20,
    edgecolors="black"
)
plt.axis("off")
plt.show()


<span style=font-size:14px>

- Based on theoretical knowledge and literature, we construct the DAG above and we will only adjust for age and sex as confounders in the model.

In [ ]:
formula_dag = (
    "Recurred_01 ~ C(Smoking_status) + C(Age_category) + C(Gender)"
)

log_model_dag = smf.logit(formula=formula_dag, data=tcr_log).fit()
log_model_dag.summary()

In [ ]:
params = log_model_dag.params
conf = log_model_dag.conf_int()

conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']

conf['p-value'] = log_model_dag.pvalues

results = conf.copy()
results[['2.5%', '97.5%', 'OR']] = np.exp(results[['2.5%', '97.5%', 'OR']])

print(results.round(3))


<span style=font-size:14px>
In this DAG based model, ever smoking "causes" recurrence of thyroid cancer, and people who ever smoked had 2 times the odds of recurrencecompared to those who never smoked, adjusting for age and gender (Adj OR= 2.036, 95% CI = [1.028, 4.031 ]). 

The model performed better than the intercept only model, with a Log-Likelihood of -194.54 versus -227.82 for the null. The pseudo R² (McFadden’s R²) of 0.146 suggest however, a moderate effect size, and the model explains about 14.6% of the variation in recurrence.

### 4.1.4 Adjusting for other predictors - stepwise approach

<span style=font-size:14px>

- Model 1: recurrence as a function of smoking status, controlled for age category, gender, thyroid function , physical examination, and adenopathy.
  

In [ ]:
formula1 = (
    "Recurred_01 ~ C(Smoking_status) + C(Age_category) + C(Gender) + " 
    "C(Thyroid_01) + C(Physical_01) + C(Adenopathy_01)"     
)

log_model1 = smf.logit(formula=formula1, data=tcr_log).fit()
log_model1.summary()

<span style=font-size:14px>
Model fit:

- Pseudo R-squared (McFadden’s R²) = 0.4397 → The predictors account for a ~44% improvement in model fit over an intercept-only model, based on McFadden’s pseudo R².

- Log-Likelihood (LL) = -127.64 vs LL-Null = -227.82 → The fitted model improves the log-likelihood considerably over a null model (intercept-only).

- Likelihood Ratio (LR) test p-value = 1.6 × 10⁻⁴⁰ → Extremely significant, confirming that the model as a whole fits the data much better than a null model.

In [ ]:
params = log_model1.params
conf = log_model1.conf_int()

conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']

conf['p-value'] = log_model1.pvalues

results = conf.copy()
results[['2.5%', '97.5%', 'OR']] = np.exp(results[['2.5%', '97.5%', 'OR']])

print(results.round(3))


<span style="font-size:14px">

In this adjusted model, having a positive smoking history (ever smoked) was not associated with higher odds of thyroid cancer recurrence compared to never smokers (p = 0.329), after adjusting for other factors. This finding is consistent with the current literature, which has not established smoking as a significant risk factor for thyroid cancer or its recurrence[^1]. To the contrary, some studies have reported an inverse relationship between smoking and thyroid cancer, although such conclusions should be interpreted with caution[^2].  

We also noted separation issues with some variables, such as adenopathy and, to a lesser extent, age category. Therefore, adenopathy will not be included in subsequent models.  

[^1]: Jiang H, Li Yi, Shen J, et al. *Cigarette smoking and thyroid cancer risk: A Mendelian randomization study*. Cancer Med. 2023;12:19866–19873. doi:10.1002/cam4.6570  
[^2]: Lee JH, Chai YJ, Yi KH. *Effect of cigarette smoking on thyroid cancer: Meta-analysis*. Endocrinol Metab (Seoul). 2021 Jun;36(3):590–598. doi:10.3803/EnM.2021.954. PMID: 34034364; PMCID: PMC8258339.  
</span>


<span style=font-size:14px>

- Model 2: recurrence as a function of smoking status, controlled for age category, gender, thyroid function , physical examination, and history of radiotherapy.


In [ ]:
formula2 = (
    "Recurred_01 ~ C(Smoking_status) + C(Age_category) + C(Gender) + " 
    "C(Thyroid_01) + C(Physical_01) + C(Adenopathy_01) + C(Hx_Radiotherapy)"   
)

log_model2 = smf.logit(formula=formula2, data=tcr_log).fit()
log_model2.summary()

<span style=font-size:14px>
Model fit:
adding focality did not improve the model fit compared to model 1 (with 0.4433 and LLR of -126.82).

In [ ]:
params = log_model2.params
conf = log_model2.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']

conf['p-value'] = log_model2.pvalues

results = conf.copy()
results[['2.5%', '97.5%', 'OR']] = np.exp(results[['2.5%', '97.5%', 'OR']])

print(results.round(3))

<span style="font-size:14px">

We still don't find a significant association between ever smoking and thyroid cancer recurrence, adjusting for the other selected predictors. 

<span style=font-size:14px>

- Model 3: recurrence as a function of smoking status, controlled for age category, gender, thyroid function , physical examination, history of radiotherapy, focality and pathology.

In [ ]:
formula3 = (
    "Recurred_01 ~ C(Smoking_status) + C(Age_category) + C(Gender) + " 
    "C(Thyroid_01) + C(Physical_01) + C(Adenopathy_01) + C(Hx_Radiotherapy) + C(Focality) + " 
    "C(Pathology)"   
)

log_model3 = smf.logit(formula=formula3, data=tcr_log).fit()
log_model3.summary()

In [ ]:
params = log_model3.params
conf = log_model3.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']

conf['p-value'] = log_model3.pvalues

results = conf.copy()
results[['2.5%', '97.5%', 'OR']] = np.exp(results[['2.5%', '97.5%', 'OR']])

print(results.round(3))

<span style=font-size:14px>
Still no significant association between ever smoking and thyroid cancer recurrence, adjusting for the other selected predictors.

### 4.1.5 Conclusion

In [ ]:
models = {
    "Model 1": log_model_dag,
    "Model 2": log_model1,
    "Model 3": log_model2,
    "Model 4": log_model3
    

}

summary_data = []

for name, model in models.items():
    # number of predictors = Df Model
    df_model = int(model.df_model)
    # number of estimated parameters (predictors + intercept)
    k = int(df_model + 1)
    # log-likelihood
    ll = model.llf
    # pseudo R²
    pseudo_r2 = model.prsquared
    # AIC and BIC (available directly)
    aic = model.aic
    bic = model.bic
    
    summary_data.append([name, df_model, ll, pseudo_r2, aic, bic])

# Create DataFrame
summary_table = pd.DataFrame(summary_data, 
                             columns=["Model", "Predictors", "Log-Likelihood", 
                                      "Pseudo R² (McFadden)", "AIC", "BIC"])

print(summary_table.round(3))


<span style=font-size:14px>

In conclusion:

- Model 1 is the original DAG-based model, based on theoretical knowledge and literature. This is the only model which shows a significant association between smoking and recurrence of thyroid cancer.
- Model 2 is the most parsimonious, with 6 predictors, a Pseudo R² of 0.44, and the lowest BIC (296.9). This suggests it balances fit and simplicity well.
- Model 3 adds one more predictor but provides only a negligible improvement in fit (Pseudo R² 0.443, LL –126.8). Its AIC and BIC are worse than Model 1, so it does not justify the extra complexity.
- Model 4 includes 11 predictors and shows the best overall fit (highest Pseudo R² = 0.496, lowest AIC = 253.8). However, its BIC (301.2) is higher than Model 1, reflecting a greater penalty for complexity.

Modern epidemiological theory emphasize the need for a causal framework to guide variables selection in regression modeling. The DAG-based model (Model 1) is therefore preferred, as it is grounded in theoretical knowledge and literature, and avoids the pitfalls of stepwise selection.